## Word Embedding Analysis

In this section, we explore several approaches to generate word embeddings for the textual data in our dataset: both problem descriptions and code solutions.   
Our goal is to obtain richer and more informative representations of each document.

We proceed in the following steps:
- Train Word Embedding Models: we begin by training traditional word embedding models such as Word2Vec and FastText on our dataset.   

- Aggregate Word Embeddings into Document Representations: after obtaining the word embeddings, we compute document-level embeddings (for both descriptions and solutions) by aggregating the individual word vectors.  

### Initialization

In this first step, we load the cleaned dataset and recall the structure of our problem descriptions.
Each description may contain multiple sentences, and we aim to extract all of them to build a comprehensive corpus of individual sentences.


This cell will download the problem_and_human_solutions_list.jsonl file from the Hugging Face Hub.  
The dataset file contains problem descriptions and human solutions.

In [ ]:
from huggingface_hub import hf_hub_download

file_path_1 = hf_hub_download(
    repo_id="facebook/BigOBench", # Repository containing the dataset
    filename="data/problem_and_human_solutions_list.jsonl", # File of interest
    repo_type="dataset"  # Specifies this is a dataset repository
)

c:\Users\utente\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We parse the .jsonl file line by line (in order to avoid problems with RAM).

In [2]:
import json
 
df_problem_and_human_solutions_list = []
with open(file_path_1, "r", encoding="utf-8") as f:
    for line in f:
        try:
            df_problem_and_human_solutions_list.append(json.loads(line))
        except json.JSONDecodeError as e:
            print("Erorr JSON:", e)
 
print("Number of rows", len(df_problem_and_human_solutions_list))

Number of rows 3105


Convert the list of dictionaries into a DataFrame showing some rows.

In [ ]:
import pandas as pd
df_problem_and_human_solutions_list = pd.DataFrame(df_problem_and_human_solutions_list)
df_problem_and_human_solutions_list.head()

,problem_id,problem_name,description,correct_solution_list,data_source,source_specific_limits,codeforces_specific_metadata,tests,human_accuracy_rate,dataclass,complexity_framework
0,0,339_C. Xenia and Weights,{'text': 'Xenia has a set of weights and pan s...,"[{'solution_id': '0_0', 'solution_code': '__au...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 339, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '0000000101 3 ', '...",0.281633,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.40579710144927...
1,1,1547_E. Air Conditioners,{'text': 'On a strip of land of length n there...,"[{'solution_id': '1_0', 'solution_code': 'def ...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 1547, 'cf_index': 'E', 'cf_p...",{'public_tests': [{'input': '5 6 2 2 5 14 16 ...,0.620833,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.11409395973154...
2,2,268_C. Beautiful Sets of Points,{'text': 'Manao has invented a new mathematica...,"[{'solution_id': '2_0', 'solution_code': 'if _...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 268, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '2 2 ', 'output': ...",0.525066,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.05527638190954...
3,3,478_C. Table Decorations,"{'text': 'You have r red, g green and b blue b...","[{'solution_id': '3_0', 'solution_code': 'a = ...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 478, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '1 1 1 ', 'output'...",0.562264,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.41610738255033...
4,4,5_C. Longest Regular Bracket Sequence,{'text': 'This is yet another problem dealing ...,"[{'solution_id': '4_0', 'solution_code': 'stri...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 5, 'cf_index': 'C', 'cf_poin...","{'public_tests': [{'input': ')((())))(()()) ',...",0.395939,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.03846153846153...


Download the second file containing light version of complexity labels.   
We will use it later when we will need the code complexities

In [41]:
file_path_2 = hf_hub_download(
    repo_id="facebook/BigOBench",
    filename="data/complexity_labels_light.jsonl",
    repo_type="dataset"
)

Parse the complexity labels file as before line by line.

In [42]:
import json
 
df_complexity_labels_light = []
with open(file_path_2, "r", encoding="utf-8") as f:
    for line in f:
        try:
            df_complexity_labels_light.append(json.loads(line))
        except json.JSONDecodeError as e:
            print("Erorr JSON:", e)
 
print("Number of rows", len(df_complexity_labels_light))

Number of rows 1190250


Convert label list to DataFrame showing some rows.

In [43]:
import pandas as pd
df_complexity_labels_light_we = pd.DataFrame(df_complexity_labels_light)
df_complexity_labels_light_we.head()

,problem_id,problem_name,solution_id,time_complexity_inferred,space_complexity_inferred,time_curve_coefficient,space_curve_coefficient
0,0,339_C. Xenia and Weights,0_0,O(1),O(n**2),0.000019,0.012433
1,0,339_C. Xenia and Weights,0_1,None,None,NaN,NaN
2,0,339_C. Xenia and Weights,0_2,O(1),O(1),0.000004,584.000000
3,0,339_C. Xenia and Weights,0_3,None,None,NaN,NaN
4,0,339_C. Xenia and Weights,0_4,O(1),O(1),0.000013,323.000000


In [ ]:
# Importing necessary libraries
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import string
import re
import seaborn as sns
import pandas as pd
import numpy as np
import collections
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk import FreqDist

regex_punctuation = '[' + string.punctuation + ']'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


As we have already seen there are some problems that have their description in other languages, just remove them.

In [ ]:
import json
# If the 'description' column contains JSON strings, decode them into Python dictionaries
if isinstance(df_problem_and_human_solutions_list['description'].iloc[0], str):
    df_problem_and_human_solutions_list['description'] = df_problem_and_human_solutions_list['description'].apply(json.loads)

# Filter rows where the description has been marked as translated
filtered_df = df_problem_and_human_solutions_list[df_problem_and_human_solutions_list['description'].apply(
    lambda x: isinstance(x, dict) and x.get('is_description_translated') is True
)]

print(f"Number of rows where description['is_description_translated'] is True: {len(filtered_df)}")

Number of rows where description['is_description_translated'] is True: 29


In [7]:
# Remove from description is_description_translated and untranslated_text
def remove_untranslated_text(description):
    if isinstance(description, dict):
        description.pop('is_description_translated', None)
        description.pop('untranslated_text', None)
    return description

df_problem_and_human_solutions_list['description'] = df_problem_and_human_solutions_list['description'].apply(remove_untranslated_text)

Remove dict in description and keep only the text

In [ ]:
def extract_text_from_description(description):
    if isinstance(description, dict):
        return description.get('text', '')
    return description

df_problem_and_human_solutions_list['description'] = df_problem_and_human_solutions_list['description'].apply(extract_text_from_description)

In [ ]:
# Visualize last changes in the dataset
df_problem_and_human_solutions_list.head()

,problem_id,problem_name,description,correct_solution_list,data_source,source_specific_limits,codeforces_specific_metadata,tests,human_accuracy_rate,dataclass,complexity_framework
0,0,339_C. Xenia and Weights,Xenia has a set of weights and pan scales. Eac...,"[{'solution_id': '0_0', 'solution_code': '__au...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 339, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '0000000101 3 ', '...",0.281633,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.40579710144927...
1,1,1547_E. Air Conditioners,On a strip of land of length n there are k air...,"[{'solution_id': '1_0', 'solution_code': 'def ...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 1547, 'cf_index': 'E', 'cf_p...",{'public_tests': [{'input': '5 6 2 2 5 14 16 ...,0.620833,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.11409395973154...
2,2,268_C. Beautiful Sets of Points,Manao has invented a new mathematical term — a...,"[{'solution_id': '2_0', 'solution_code': 'if _...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 268, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '2 2 ', 'output': ...",0.525066,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.05527638190954...
3,3,478_C. Table Decorations,"You have r red, g green and b blue balloons. T...","[{'solution_id': '3_0', 'solution_code': 'a = ...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 478, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '1 1 1 ', 'output'...",0.562264,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.41610738255033...
4,4,5_C. Longest Regular Bracket Sequence,This is yet another problem dealing with regul...,"[{'solution_id': '4_0', 'solution_code': 'stri...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 5, 'cf_index': 'C', 'cf_poin...","{'public_tests': [{'input': ')((())))(()()) ',...",0.395939,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.03846153846153...


In this steps, we prepare the text data for word embedding training, by extracting and splitting the problem descriptions into individual sentences.

Specifically, we:
- Create a copy of the DataFrame to work with.
- Extract the translated and cleaned description texts.

In [ ]:
df_problem_and_human_solutions_list_we = df_problem_and_human_solutions_list.copy()
descriptions= df_problem_and_human_solutions_list_we['description'].dropna()

# Print the first 10 descriptions to have a look at the data
print("First 10 descriptions:")
print(descriptions[0:10])

0    Xenia has a set of weights and pan scales. Eac...
1    On a strip of land of length n there are k air...
2    Manao has invented a new mathematical term — a...
3    You have r red, g green and b blue balloons. T...
4    This is yet another problem dealing with regul...
5    Consider the infinite sequence of integers: 1,...
6    Let's introduce a number system which is based...
7    A few years ago Sajjad left his school and reg...
8    Polycarp likes to play with numbers. He takes ...
9    There are N people standing in a queue from we...
Name: description, dtype: object


- Remove newline characters and split the text into sentences based on punctuation using regular expressions.

In [ ]:
# Separating sentences from descriptions using regex
descriptions= [re.sub(r'\n', ' ', description) for description in descriptions]
descriptions_sentences = [re.split(r'[.!?]\s', description) for description in descriptions]

# Printing the sentences of the first description
descriptions_sentences[0]

['Xenia has a set of weights and pan scales',
 'Each weight has an integer weight from 1 to 10 kilos',
 'Xenia is going to play with scales and weights a little',
 'For this, she puts weights on the scalepans, one by one',
 'The first weight goes on the left scalepan, the second weight goes on the right scalepan, the third one goes on the left scalepan, the fourth one goes on the right scalepan and so on',
 'Xenia wants to put the total of m weights on the scalepans',
 ' Simply putting weights on the scales is not interesting, so Xenia has set some rules',
 'First, she does not put on the scales two consecutive weights of the same weight',
 'That is, the weight that goes i-th should be different from the (i + 1)-th weight for any i (1 ≤ i < m)',
 'Second, every time Xenia puts a weight on some scalepan, she wants this scalepan to outweigh the other one',
 'That is, the sum of the weights on the corresponding scalepan must be strictly greater than the sum on the other pan',
 ' You are g

- Flatten the list of lists into a single list of individual sentences.

In [ ]:
from pandas.core.common import flatten

sentences = list(flatten(descriptions_sentences))

# Show the first 30 sentences of the all descriptions corpus
# No distinction between documents anymore (just a list of sentences)
sentences[0:30]

['Xenia has a set of weights and pan scales',
 'Each weight has an integer weight from 1 to 10 kilos',
 'Xenia is going to play with scales and weights a little',
 'For this, she puts weights on the scalepans, one by one',
 'The first weight goes on the left scalepan, the second weight goes on the right scalepan, the third one goes on the left scalepan, the fourth one goes on the right scalepan and so on',
 'Xenia wants to put the total of m weights on the scalepans',
 ' Simply putting weights on the scales is not interesting, so Xenia has set some rules',
 'First, she does not put on the scales two consecutive weights of the same weight',
 'That is, the weight that goes i-th should be different from the (i + 1)-th weight for any i (1 ≤ i < m)',
 'Second, every time Xenia puts a weight on some scalepan, she wants this scalepan to outweigh the other one',
 'That is, the sum of the weights on the corresponding scalepan must be strictly greater than the sum on the other pan',
 ' You are g

These sentences will be the basis for training embedding models like Word2Vec or FastText.

### Tokenization of Descriptions

In this part, the main goal is to obtain an effective tokenization of each sentence in our dataset.  

We prepare the corpus by applying the following preprocessing steps to each sentence using a custom tokenizer function:

- Removing unwanted punctuation and characters while preserving programming symbols (e.g., +, -, =, [], ())

- Normalizing Unicode math symbols (e.g., ≤, →) into their ASCII equivalents (e.g., <=, ->)

- Replacing complex numeric formats with general placeholders (e.g., sci_num, int_num, frac_num)

- Spacing out code symbols to treat them as individual tokens (e.g., f(x) → f ( x ))

- Lemmatizing words using spaCy

- Removing natural language stopwords while preserving programming-relevant terms (e.g., if, return, while)

- Filtering out:
    - Words that are too long (likely noise)
    - Unwanted sequences with repeated characters or excessive dots

In [ ]:
# Install spaCy and its English language model
#!pip install spacy
#!python -m spacy download en_core_web_sm

In [ ]:
import re
import spacy
from gensim.models import Word2Vec
from typing import List
from nltk.corpus import stopwords
import pickle

# Load spaCy English model (make sure it's installed: python -m spacy download en_core_web_sm)
nlp = spacy.load("en_core_web_sm")

# Custom stopword list (exclude important programming-related words)
custom_stopwords = set(stopwords.words('english')) - {
    'if', 'for', 'while', 'function', 'input', 'output', 'return', 'else', 'and', 'or','not','each','all','any','to',
    'not', 'in', 'is', 'as', 'def', 'import', 'from', 'class', 'with', 'try', 'except',
    'i', 'j', 'k', 'x', 'y', 'z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'l', 'm', 'n',
    's', 't', 'u', 'v', 'w', 'p', 'q', 'r', 'yes', 'no', 'true', 'false',
}
custom_stopwords = set([word.lower() for word in custom_stopwords])

# Define mapping of Unicode math/symbol characters to ASCII equivalents
UNICODE_SYMBOLS = {
    '≤': '<=',
    '≥': '>=',
    '≠': '!=',
    '→': '->',
    '←': '<-',
    '⋅': '*',
    '×': '*',
    '÷': '/',
    '−': '-',
    '“': '"',
    '”': '"',
    '‘': "'",
    '’': "'"
}

# Function that replace Unicode math/special symbols with ASCII equivalents.
def normalize_unicode_symbols(text: str) -> str:
    for uni_char, ascii_equiv in UNICODE_SYMBOLS.items():
        text = text.replace(uni_char, f' {ascii_equiv} ')
    return text

# Function that replace complex numbers with labeled placeholders.
def substitute_special_numbers(text: str) -> str:
    # Replace scientific notation (e.g., 1e9, -2.5E-10)
    text = re.sub(r"\b-?\d+(\.\d+)?[eE][+-]?\d+\b", " sci_num ", text)
    # Replace decimal numbers (e.g., 3.14, -0.5)
    text = re.sub(r"\b-?\d+\.\d+\b", " float_num ", text)
    # Replace exponent notation (e.g., 10^5)
    text = re.sub(r"\b\d+\^\d+\b", " exp_num ", text)
    # Replace fractions (e.g., 3/4)
    text = re.sub(r"\b-?\d+/\d+\b", " frac_num ", text)
    # Replace integers (keep this last to not override previous ones)
    text = re.sub(r"\b-?\d+\b", " int_num ", text)
    return text

The tokenizer function performs a sequence of preprocessing steps (already described previously) that are essential for effective embedding.   

The tokenizer integrates all previously defined functions, and adds new regular expression filters to remove or replace some undesirable words from sentences.

In [ ]:

def custom_tokenize(sentence: str) -> List[str]:
    
    # 1. Lowercase
    sentence = sentence.lower()
    
    # 2.Normalize Unicode symbols (e.g., ≤, ≥, →) to ASCII equivalents
    sentence = normalize_unicode_symbols(sentence)

    # 3. Preserve important math/programming symbols (remove others)
    # Keep: +, -, *, /, =, <, >, [, ], (, ), ., _, ,, |, ^ numbers and letters
    sentence = re.sub(r"[^\w\s\[\]\(\)\+\-\*/=<>.,\{\}\|\^]", "", sentence)

    # 4. Replace complex numbers BEFORE spaCy
    sentence = substitute_special_numbers(sentence)

     # Remove very long tokens and multiple dots
    sentence = re.sub(r"\.{2,}", " ", sentence)

    sentence = re.sub(r"\b\w{21,}\b", " ", sentence)
    
    # Remove tokens with 5 or more repeated characters (e.g., "aaaaabbbbbb")
    # We decided to keep this line commented out as it may remove important tokens
    #sentence = re.sub(r"\b(\w*([a-zA-Z])\2{4,}\w*)\b", " ", sentence)

    # 5. Separate symbols with space to treat them as separate tokens
    # E.g., f(x) → f ( x ), arr[2] → arr [ 2 ]
    sentence = re.sub(r"([\[\]\(\)\+\-\*/=.,\{\}\|])", r" \1 ", sentence)

    sentence = re.sub(r"\s+", " ", sentence)  # Remove extra spaces

    # 6. Tokenize using spaCy (preserves useful structures)
    doc = nlp(sentence)
    tokens = []

    for token in doc:
        # Skip punctuation and unwanted stopwords (unless technical)
        if token.text in custom_stopwords or token.is_space:
            continue

        # Lemmatize token using spaCy

        lemma = token.lemma_.lower()

        # Optional: normalize common variable names (e.g., a_1, res, val)
        # We decide to keep them as they are
        if re.fullmatch(r"[a-zA-Z_]+\d*", lemma):
            # You can optionally map them to <var> or leave them
            tokens.append(lemma)
        else:
            tokens.append(lemma)

    return tokens

This custom tokenizer produces clean, structured, and semantically meaningful tokens, which can significantly enhance the quality of the resulting embeddings.   
Here an example to have an idea of how it works with test sentences.

In [ ]:

# Example input: technical sentences with variables, functions, arrays and other bad tokens
sentences_trial = [
    "The function f(x) = x^2 + 10000000000000000000000000000, define f(x) as x * x + 1 to 100",
    "The function f(x) = x^2 + sjndjncjd0000000000000000000000000, define f(x) as x * x + 1 nskkk^^^^^sk1 naallloooonnn ... .. .... kk lll_test",
    "The function f(x) returns x * x + 1",
    "Use arr[0] and arr[1] to store the inputs.",
    "Check if a_1 + b_2 == c_3.",
    "Return the result in res after the while loop.",
    "for i in range(1, 10): sum += arr[i]",
    "Initialize matrix m with zeros using numpy.zeros((3,3))",
    "inputs = [1, 2, 3]; output = f(inputs[0])",
    "Sort the array nums in ascending order.",
    "Let temp = (x + y) / 2",
    "Function g(n) = n*(n-1)/2 calculates combinations.",
    "temperature in cell 1 is: min(14 + |2 - 1|, 16 + |5 - 1|)=min(14 + 1, 16 + 4)=min(15, 20)=15;    2",
    "))(skdfksf + u) -> 0.5 * (a + b) + 1.5 * (c - d) / e; 3.14 * pi = 3.14;",
    "if (x > 0) and (y < 10): print('Valid') else: print('Invalid')",
    "while (i < 10): i += 1; print(i)",
    "from math import sqrt; result = sqrt(x**2 + y**2)",
    "def calculate_area(radius): return 3.14 * radius ** 2 * 10^17",
    "The first line contains two integers n (1 ≤ n ≤ 3 ⋅ 10^5) and k (1 ≤ k ≤ n) — the length of the strip of land and the number of air conditioners respectively'",
    "The second line contains k integers a_1, a_2, …, a_k (1 ≤ a_i ≤ n) — positions of air conditioners on the strip of land",
    "The third line contains k integers t_1, t_2, …, t_k (1 ≤ t_i ≤ 10^9) — temperatures of air conditioners",
    "It is guaranteed that the sum of n over all test cases does not exceed 3 ⋅ 10^5",
    "Output  For each test case output n integers separated by space: temperatures of air in cells",
    "Example  Input   5  6 2 2 5 14 16  10 1 7 30  5 5 3 1 4 2 5 3 1 4 2 5  7 1 1 1000000000",
    "Manao has invented a new mathematical term — a beautiful set of points",
    "Examples  Input  0000000101 3   Output  YES 8 10 8   Input  1000000000 2   Output  NO",
    "For each cell i (1 ≤ i ≤ n) find it's temperature, that can be calculated by the formula $$$min_{1 ≤ j ≤ k}(t_j + |a_j - i|),$$$  where |a_j - i| denotes absolute value of the difference a_j - i"
]

# Apply custom tokenizer
tokenized_corpus_trial= [custom_tokenize(sent) for sent in sentences_trial]

# Print tokenized output
for i, toks in enumerate(tokenized_corpus_trial):
    print(f"Sentence {i+1}: {sentences_trial[i]} \n tokens: {toks}")

Sentence 1: The function f(x) = x^2 + 10000000000000000000000000000, define f(x) as x * x + 1 to 100 
 tokens: ['function', 'f', '(', 'x', ')', '=', 'x^', 'int_num', '+', 'int_num', ',', 'define', 'f', '(', 'x', ')', 'as', 'x', '*', 'x', '+', 'int_num', 'to', 'int_num']
Sentence 2: The function f(x) = x^2 + sjndjncjd0000000000000000000000000, define f(x) as x * x + 1 nskkk^^^^^sk1 naallloooonnn ... .. .... kk lll_test 
 tokens: ['function', 'f', '(', 'x', ')', '=', 'x^', 'int_num', '+', ',', 'define', 'f', '(', 'x', ')', 'as', 'x', '*', 'x', '+', 'int_num', 'nskkk^^^^^sk1', 'naallloooonnn', 'kk', 'lll_test']
Sentence 3: The function f(x) returns x * x + 1 
 tokens: ['function', 'f', '(', 'x', ')', 'return', 'x', '*', 'x', '+', 'int_num']
Sentence 4: Use arr[0] and arr[1] to store the inputs. 
 tokens: ['use', 'arr', '[', 'int_num', ']', 'and', 'arr', '[', 'int_num', ']', 'to', 'store', 'input', '.']
Sentence 5: Check if a_1 + b_2 == c_3. 
 tokens: ['check', 'if', 'a_1', '+', 'b_2', '='

It's easy to see that it's able to split the sentence in a reasonable and clearer way.

We now apply the previously defined custom_tokenize() function to the entire sentence corpus derived from the problem descriptions.
- Each sentence is transformed into a list of cleaned and lemmatized tokens, preserving meaningful syntax and semantics, especially relevant to programming.

- We then print out the tokenization results for the first 20 sentences to inspect the quality and structure of the tokens.

In [ ]:
# Apply custom tokenizer on real data
tokenized_corpus = [custom_tokenize(sent) for sent in sentences]

# Print tokenized output
for i, toks in enumerate(tokenized_corpus):
    if i >= 20:  # Limit to first 20 sentences for brevity
        break
    print(f"Sentence {i+1}: {sentences[i]} \n tokens: {toks}")


Sentence 1: Xenia has a set of weights and pan scales 
 tokens: ['xenia', 'a', 'set', 'weight', 'and', 'pan', 'scale']
Sentence 2: Each weight has an integer weight from 1 to 10 kilos 
 tokens: ['each', 'weight', 'integer', 'weight', 'from', 'int_num', 'to', 'int_num', 'kilo']
Sentence 3: Xenia is going to play with scales and weights a little 
 tokens: ['xenia', 'be', 'go', 'to', 'play', 'with', 'scale', 'and', 'weight', 'a', 'little']
Sentence 4: For this, she puts weights on the scalepans, one by one 
 tokens: ['for', ',', 'put', 'weight', 'scalepans', ',', 'one', 'one']
Sentence 5: The first weight goes on the left scalepan, the second weight goes on the right scalepan, the third one goes on the left scalepan, the fourth one goes on the right scalepan and so on 
 tokens: ['first', 'weight', 'go', 'left', 'scalepan', ',', 'second', 'weight', 'go', 'right', 'scalepan', ',', 'third', 'one', 'go', 'left', 'scalepan', ',', 'fourth', 'one', 'go', 'right', 'scalepan', 'and']
Sentence 6: X

Save the tokenized corpus into a pickle file to avoid to do the long computation each time.

In [ ]:
# Save the tokenized corpus to a file, it's a list of lists
with open("tokenized_descriptions_corpus.pkl", "wb") as f:
    pickle.dump(tokenized_corpus, f)

Read the tokenized description corpus already stored in a pickle file.

In [26]:
with open("tokenized_descriptions_corpus.pkl", "rb") as f:
    tokenized_corpus = pickle.load(f)

In [ ]:
#!pip install gensim

### Word2Vec

In this section, we train a Word2Vec model using the tokenized and preprocessed corpus of problem descriptions.  

We use the Skip-gram architecture (sg=1), which is known to perform better on infrequent words, a desirable property for our technical dataset where specific terms might appear rarely but hold significant meaning.

In [ ]:
from gensim.models import Word2Vec

# Train Word2Vec model
w2v_model = Word2Vec(
    sentences=tokenized_corpus,  # tokenized corpus is a list of lists of token (already created by your previous code)
    vector_size=100,    # dimension of the word vectors
    window=5,           # context window size
    min_count=2,        # filter out words with frequency lower than 2
    workers=4,          # number of threads used for training
    sg=1                # 1 = skip-gram, 0 = CBOW
)

# Save the model
w2v_model.save("word2vec.model")


Useful if we want to skip the training having already computed the result

In [ ]:
# Load the model
w2v_model = Word2Vec.load("word2vec.model")

Print the vocabulary keys and its size

In [162]:
# Vocabulary size
print(w2v_model.wv.key_to_index.keys())
print("Number of words in the vocabulary:", len(w2v_model.wv.key_to_index))

dict_keys(['int_num', ',', 'a', '=', 'be', 'in', 'and', '<', '(', ')', 'input', 'n', 'output', '-', 'number', 'integer', 'line', '*', 'first', 'contain', 'example', 'one', 'if', 'for', 'i', 'each', 'print', 'from', 'not', '+', 'second', 'all', 'b', 'with', 'string', 'two', 'give', 'case', 'x', 's', 'test', '{', '}', 'k', 'th', 'as', 'no', 'note', 'or', '.', ']', '[', 't', '>', 'exp_num', 'any', 'possible', '|', 'follow', 'length', 'm', 'single', 'leq', 'array', 'sequence', 'time', 'answer', '_', 'find', 'element', 'equal', 'value', 'letter', 'yes', 'consist', 'character', 'a_i', 'y', 'day', 'operation', 'get', 'move', 'make', 'order', 'minimum', 'c', 'want', 'point', 'j', 'choose', 'sum', 'third', 'maximum', 'sample', 'exactly', '/', 'd', 'need', 'image', 'game', 'use', 'write', 'r', 'three', 'digit', 'cell', 'way', 'problem', 'l', 'take', 'pair', 'a_1', 'next', 'positive', 'guarantee', 'position', 'constraint', 'format', 'ai', 'right', 'go', 'otherwise', 'p', 'total', 'a_2', 'row', 'w

The model is now trained and allows us to query word similarities based on the learned embeddings.   

We retrieve the top 10 words that the model considers most semantically or contextually similar to different domain-relevant terms ("fraction","array","graph" and "matrix")   
We can interpret these as terms that frequently appear in similar contexts in the original descriptions.

In [163]:
w2v_model.wv.most_similar('fraction', topn=10)

[('denominator', 0.8700506091117859),
 ('irreducible', 0.8617052435874939),
 ('numerator', 0.8451979160308838),
 ('notation', 0.7737084627151489),
 ('primitive', 0.7539716362953186),
 ('equation', 0.7502864003181458),
 ('^x', 0.7380940914154053),
 ('quotient', 0.7311969995498657),
 ('modular', 0.7296258211135864),
 ('formula', 0.7195370197296143)]

In [36]:
w2v_model.wv.most_similar('array', topn=10)

[('subarray', 0.7538277506828308),
 ('permutation', 0.7525390386581421),
 ('element', 0.7008444666862488),
 ('subsegment', 0.6911046504974365),
 ('multiset', 0.671673595905304),
 ('sequence', 0.6378703117370605),
 ('sorted', 0.6317214965820312),
 ('mex', 0.6302142143249512),
 ('yield', 0.6286007165908813),
 ('progression', 0.6281257271766663)]

In [38]:
w2v_model.wv.most_similar('graph', topn=10)

[('undirected', 0.8931739330291748),
 ('connected', 0.8270784616470337),
 ('edge', 0.824512243270874),
 ('rooted', 0.7907810807228088),
 ('component', 0.7904513478279114),
 ('tree', 0.776930034160614),
 ('subgraph', 0.7757490873336792),
 ('root', 0.7704923152923584),
 ('unweighted', 0.769507884979248),
 ('vertex', 0.7576366066932678)]

In [39]:
w2v_model.wv.most_similar('matrix', topn=10)

[('grid', 0.7598245739936829),
 ('chessboard', 0.7413293123245239),
 ('vector', 0.7187368869781494),
 ('histogram', 0.7009692788124084),
 ('board', 0.683229386806488),
 ('configuration', 0.6795511841773987),
 ('diagonal', 0.6716492772102356),
 ('table', 0.6689319610595703),
 ('column', 0.6681662201881409),
 ('numbering', 0.6663357615470886)]

The training seems to be effective considering the previous examples.   
It shows almost always semantically close words.

One notable limitation of Word2Vec is its inability to handle out-of-vocabulary (OOV) words, those that were not seen during training or filtered out due to low frequency (min_count).  

As we can observe here (e.g., for the token 'c++'), if a word is missing from the model's vocabulary, Word2Vec cannot generate an embedding for it.   
This is problematic when working with new, unseen sentences.

In [170]:
if(w2v_model.wv.key_to_index.get('c++') is not None):
    w2v_model.wv.most_similar('c++', topn=10)
else:
    print("'c++' not in vocabulary of Word2Vec model")

'c++' not in vocabulary of Word2Vec model


In [ ]:
#!pip install -U scikit-learn
#!pip install plotly


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Visualizing Word Embeddings with t-SNE

To explore the semantic structure captured by the Word2Vec model, we randomly sampled 500 words from the learned vocabulary and projected their embeddings into a 3D space using t-SNE (t-distributed Stochastic Neighbor Embedding).

This technique is particularly effective at preserving local neighborhood structures, making it useful for visualizing how similar words cluster together in the vector space.
- Each point in the 3D scatter plot corresponds to a word vector.
- Nearby points indicate semantic or contextual similarity as learned by the model during training.

In [164]:
import random

sample = random.sample(list(w2v_model.wv.key_to_index), 500)
print(sample)

['deny', 'participate', 'bf', 'harshad', 'aaaaaaaaaa', 'paintbrush', 'argument', 'origin', 'normal', 'gram', 'arthur', 'fast', 'fruit', 'film', 'divisibility', 'insertion', 'elderly', 'dec_2', 'volga', 'y_3', 'bucket', 'desribe', 'equation', 'doggo', 'aiajak', 'unnecessary', 'plush', 'await', 'learning', '1000_2', 's_v', 'criterion', 'unmarked', 'grandfather', 'calorie', 'circumstance', 'irrespective', 'mex', 'substre', 'caterpillar', 'jacket', 'a_t', 'y^x', 'andrew', 'presence', 'factorial', 'occupation', 'special', 'adventure', 'max2', 'abcdfdcba', 'reveal', 'editor', 'ripen', 'bbbbb', 'unknown', 'earth', 'zzzaa', 'scrooge', 'specify', 'agasa', 'fastcode', 'bac', 'corner', 'reach', 'boat', 'aqi', 'iv', 'leesin', 'worth', 'konjac', 'tak', 'mitya', 'dachshund', 'diversity', 'lj', 'ada', 'force', 'stretch', 'puzzle', 'hu', 'irreducible', 'respectively', 'aabbcc', 'seater', 'shinkansen', 'counter', 'qip', 'theme', 'doctor', 'ascending', '[', 'satisfy', 'lena', 'current', 'wall', 'catacom

Here we are obtaining the embedding vectors for each of the previously randomly sampled words.

In [ ]:
word_vectors = w2v_model.wv[sample]
# Show the vector structure
print(word_vectors)
# Show the shape of the word vectors 
print("Shape of word vectors:", word_vectors.shape)

[[-0.2165097   0.23865132  0.1422642  ... -0.07689774 -0.10238189
  -0.00671862]
 [ 0.0646564  -0.03404604  0.0800567  ... -0.14391412 -0.3691363
  -0.01748073]
 [-0.23485583  0.17882562 -0.01770534 ... -0.29749477  0.04452924
  -0.01693278]
 ...
 [-0.13107689  0.17704591  0.07094711 ... -0.1156648  -0.06620975
   0.04590727]
 [-0.24537122  0.18454692  0.07825024 ... -0.12724    -0.0672618
   0.08927469]
 [-0.30973276  0.09563615  0.36001012 ...  0.07332251 -0.09852687
   0.21584803]]
Shape of word vectors: (500, 100)


Here we have an initial visualization using plotly

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import plotly.express as px

tsne = TSNE(n_components=3, n_iter=2000)
tsne_embedding = tsne.fit_transform(word_vectors)

x, y, z = np.transpose(tsne_embedding)

fig = px.scatter_3d(x=x[:200],y=y[:200],z=z[:200],text=sample[:200])
fig.update_traces(marker=dict(size=3,line=dict(width=2)),textfont_size=10)
fig.show()


c:\Users\utente\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



We can hardly notice that some name of variables are put closer such us some numbers or strange words.   
Anyway, seems an hard task try to understand the quality of those embeddings.

To better understand how the embedding model (Word2Vec or FastText later) captures semantic relationships, we implemented a function that visualizes clusters of similar words centered around a predefined set of keywords.

This function:
- Takes a list of target words (e.g., "matrix", "graph", "loop") as input.
- Retrieves the top-15 most similar words for each target from the model.
- Projects the resulting word vectors into 3D space using t-SNE.
- Groups the words visually by color to highlight semantic similarity clusters around each center word.

In [ ]:

# Plot the most similar words for a given group of words using FastText o Word2Vec model.

# Parameters:
#    model : The trained FastText or W2V model.
#    centers (list): List of words to find similar words for.

def plot_word_embedding_grouped(model, centers):
    
    labels = [] # Will store the group (center word) each word belongs to
    words = []  # List of all words to visualize (centers + their similar words)
    for center in centers:
        try:
            # Retrieve top-15 most similar words to the center word
            similar_words = [w[0] for w in model.wv.most_similar(center, topn=15)]
            # Include the center word itself in the visualization
            similar_words.append(center)
            for word in similar_words:
                # Add word only if it's not already in the list
                if word not in words:
                    words.append(word)
                    labels.append(center) # Label with the corresponding center word
        except KeyError:
            # Handle the case where a center word is not in the vocabulary
            print(f"Word '{center}' not in vocabulary.")

    # Step 2: Extract word vectors from the model
    vectors = np.array([model.wv[word] for word in words])

    # Step 3: Apply t-SNE to reduce dimensionality to 3D
    tsne = TSNE(n_components=3, n_iter=2000, perplexity=10, init='random', random_state=42)
    tsne_embedding = tsne.fit_transform(vectors)
    x, y, z = np.transpose(tsne_embedding) # Unpack the 3D coordinates

    # Step 4: Create a DataFrame for visualization
    df = pd.DataFrame({
        'x': x,
        'y': y,
        'z': z,
        'word': words,
        'group': labels
    })

    # Step 5: Plot using Plotly (3D scatter)
    fig = px.scatter_3d(
        df,
        x='x',
        y='y',
        z='z',
        text='word',  # Show word label next to point
        color='group',  # Color by center word
        title='3D t-SNE Word Embedding Visualization (Grouped)'
    )

    fig.update_traces(marker=dict(size=5, line=dict(width=2)), textfont_size=10)
    fig.show()

Plotting the most similar words for a given group of words using Word2Vec model with the previous function. 

In [ ]:
centers=["sort", "search", "delete", "graph", "c++"]
plot_word_embedding_grouped(w2v_model, centers)

Word 'c++' not in vocabulary.


c:\Users\utente\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



The visualization shows clear clusters of words around their center terms, confirming that the model captures meaningful semantic relationships.

However, we also notice some irregularities. For instance, the word "search" appears slightly isolated from its expected neighbors.

This could be due to:
- Dimensionality reduction artifacts from t-SNE, which can distort distances in 3D space.
- Contextual ambiguity — words like "search" may be used in various technical contexts (e.g., graphs, arrays, strings).


### FastText

In this section, we train two FastText models using the gensim library:
- A standard model with vector_size=100 (richer representations).
- A lightweight model with vector_size=50 (faster and smaller footprint).

FastText extends Word2Vec by representing words as bags of character n-grams, allowing it to generalize better and assign a vector also to rare or unseen words. 
This is especially useful in technical or domain-specific documents like code descriptions. 

Finally, a comparison function is used to analyze how different FastText models embed some representative words by listing their most similar words.

In [ ]:
# Import FastText from gensim
from gensim.models import FastText

 Create and train a FastText model with higher vector dimensionality.  
 (skip this part if you want to load directly the already trained models from file)

In [ ]:
ft_model = FastText(
    vector_size=100,     # Embedding vector size (richer representations)
    window=5,            # Context window
    min_count=2,         # Minimum frequency threshold
    workers=8,           # Use multiple threads
    sg=1                 # Use skip-gram (1); 0 = CBOW
)
# Build the vocabulary from the tokenized corpus
ft_model.build_vocab(corpus_iterable=tokenized_corpus)

# Train the FastText model on the tokenized corpus
ft_model.train(
    corpus_iterable=tokenized_corpus,
    total_examples=len(tokenized_corpus),
    epochs=10
)
# Save the full model
ft_model.save("fasttext_descriptions.model")

Create a lightweight FastText model with smaller vectors (skip to save time)

In [ ]:
ft_model = FastText(
    vector_size=50, # Smaller vector size for efficiency
    window=5,
    min_count=2,
    workers=8,
    sg=1
)
ft_model.build_vocab(corpus_iterable=tokenized_corpus)
# Train the lightweight model
ft_model.train(
    corpus_iterable=tokenized_corpus,
    total_examples=len(tokenized_corpus),
    epochs=10
)
# Save the light model
ft_model.save("fasttext_descriptions_light.model")

Here, the code to load the models already trained in order to save time.  
(the file of the models should have been put in the same directory of the notebook)

In [ ]:
# Load the trained models
ft_model= FastText.load("fasttext_descriptions.model")
ft_model_light= FastText.load("fasttext_descriptions_light.model")

Check the shape of the learned word vectors

In [ ]:
# Check the shape of the learned word vectors
print("Shape of ft_model vectors:", ft_model.wv.vectors.shape)
print("Shape of ft_model_light vectors:", ft_model_light.wv.vectors.shape)

Shape of ft_model vectors: (7026, 100)
Shape of ft_model_light vectors: (7026, 50)


Here a function that allows us to compare the output of multiple FastText models on a list of words printing the 10 most similar for each.  
This can help identify relevant differences in the learned embeddings across our 2 FastText models.
Parameters:
- models (dict): Dictionary with model names as keys and FastText objects as values.
- word (str): The target word to find similar words for.
- topn (int): Number of top similar words to display.


In [ ]:
# Function to compare the output of multiple FastText models
def compare_models(models: dict, word: str, topn: int = 10):
    for model_name, model in models.items():
        print(f"\n=== Model: {model_name} ===")
        if word in model.wv.key_to_index:
            print(f"\nMost similar to '{word}':")
            similar = model.wv.most_similar(word, topn=topn)
            for i, (sim_word, score) in enumerate(similar, 1):
                print(f"  {i}. {sim_word} (score: {score:.4f})")
        else:
           print(f"\n'{word}' not in vocabulary of model '{model_name}'")
           # Still attempt similarity lookup — FastText can handle OOV words using subword information
           print(f"\nMost similar to '{word}':")
           similar = model.wv.most_similar(word, topn=topn)
           for i, (sim_word, score) in enumerate(similar, 1):
            print(f"  {i}. {sim_word} (score: {score:.4f})")
           

In [ ]:
# Example usage: compare embeddings of the word 'function'
models = {
    "Large FastText": ft_model,
    "Light FastText": ft_model_light
}

target_word= "function"
compare_models(models, target_word, topn=10)



=== Model: Large FastText ===

Most similar to 'function':
  1. malfunction (score: 0.9329)
  2. functionality (score: 0.8688)
  3. functional (score: 0.8478)
  4. nonfunctional (score: 0.8020)
  5. definition (score: 0.6954)
  6. translation (score: 0.6854)
  7. recursive (score: 0.6850)
  8. reduction (score: 0.6815)
  9. notion (score: 0.6814)
  10. introduction (score: 0.6796)

=== Model: Light FastText ===

Most similar to 'function':
  1. malfunction (score: 0.9210)
  2. functionality (score: 0.8730)
  3. functional (score: 0.8426)
  4. nonfunctional (score: 0.7941)
  5. recursive (score: 0.7699)
  6. aeb (score: 0.7235)
  7. notion (score: 0.7029)
  8. recurrence (score: 0.6987)
  9. we (score: 0.6963)
  10. translation (score: 0.6962)


In [46]:
target_word= "array"
compare_models(models, target_word, topn=10)


=== Model: Large FastText ===

Most similar to 'array':
  1. eqnarray (score: 0.8938)
  2. omkarray (score: 0.8558)
  3. subarray (score: 0.7873)
  4. subbarray (score: 0.7851)
  5. element (score: 0.6615)
  6. nextelement (score: 0.6498)
  7. yield (score: 0.6217)
  8. permute (score: 0.6106)
  9. permutation (score: 0.6089)
  10. arrayhead (score: 0.6068)

=== Model: Light FastText ===

Most similar to 'array':
  1. eqnarray (score: 0.9040)
  2. omkarray (score: 0.8724)
  3. subarray (score: 0.8288)
  4. subbarray (score: 0.8218)
  5. permutation (score: 0.7831)
  6. element (score: 0.7688)
  7. nextelement (score: 0.7554)
  8. permute (score: 0.7487)
  9. sequence (score: 0.7295)
  10. subsegment (score: 0.7184)


In [47]:
target_word= "graph"
compare_models(models, target_word, topn=10)


=== Model: Large FastText ===

Most similar to 'graph':
  1. polygraph (score: 0.8697)
  2. subgraph (score: 0.8669)
  3. undirected (score: 0.8320)
  4. paragraph (score: 0.8258)
  5. rooted (score: 0.7660)
  6. edge (score: 0.7636)
  7. directed (score: 0.7547)
  8. unweighted (score: 0.7446)
  9. connected (score: 0.7296)
  10. isolated (score: 0.7291)

=== Model: Light FastText ===

Most similar to 'graph':
  1. polygraph (score: 0.8994)
  2. subgraph (score: 0.8908)
  3. paragraph (score: 0.8618)
  4. undirected (score: 0.8571)
  5. edge (score: 0.8412)
  6. rooted (score: 0.8149)
  7. unweighted (score: 0.8075)
  8. connected (score: 0.7930)
  9. directed (score: 0.7863)
  10. vertex (score: 0.7861)


In [48]:
target_word= "matrix"
compare_models(models, target_word, topn=10)


=== Model: Large FastText ===

Most similar to 'matrix':
  1. vmatrix (score: 0.9585)
  2. bmatrix (score: 0.9191)
  3. pmatrix (score: 0.9038)
  4. grid (score: 0.6744)
  5. column (score: 0.6173)
  6. matroskin (score: 0.6141)
  7. square1001 (score: 0.5893)
  8. bon (score: 0.5704)
  9. eqnarray (score: 0.5622)
  10. magnitude (score: 0.5598)

=== Model: Light FastText ===

Most similar to 'matrix':
  1. vmatrix (score: 0.9518)
  2. bmatrix (score: 0.9115)
  3. pmatrix (score: 0.8926)
  4. grid (score: 0.7233)
  5. square1001 (score: 0.6782)
  6. coloring (score: 0.6721)
  7. column (score: 0.6615)
  8. square (score: 0.6579)
  9. row (score: 0.6532)
  10. pitch (score: 0.6393)


In [49]:
target_word= "unknown_word"
compare_models(models, target_word, topn=10)


=== Model: Large FastText ===

'unknown_word' not in vocabulary of model 'Large FastText'

Most similar to 'unknown_word':
  1. unknown (score: 0.9093)
  2. known (score: 0.8381)
  3. koyomi (score: 0.7664)
  4. liouzhou_101 (score: 0.7625)
  5. beforehand (score: 0.7571)
  6. quasi (score: 0.7488)
  7. unless (score: 0.7464)
  8. alexander (score: 0.7453)
  9. keione (score: 0.7403)
  10. pencilcase (score: 0.7391)

=== Model: Light FastText ===

'unknown_word' not in vocabulary of model 'Light FastText'

Most similar to 'unknown_word':
  1. unknown (score: 0.9306)
  2. known (score: 0.8734)
  3. elapse (score: 0.8314)
  4. unrecognized (score: 0.8218)
  5. unless (score: 0.8181)
  6. koyomi (score: 0.8157)
  7. pencilcase (score: 0.8039)
  8. randomaccess (score: 0.8032)
  9. guess (score: 0.7987)
  10. primarily (score: 0.7933)


The two FastText models (large and light) yield very similar results across all comparisons, consistently returning nearly identical top similar words for each target word.

#### Visualizing Word Embeddings with t-SNE

In this cell, we use the plot_word_embedding_grouped function showed in the previous cells to visualize the word embeddings learned by the FastText model for a selected list of central words ("sort", "search", "delete", "graph", and "dog"). 

Note: we use the large version of FastText model for this plot

In [175]:
# Plotting 
centers=["sort", "search", "delete", "graph", "dog"]
plot_word_embedding_grouped(ft_model, centers)

c:\Users\utente\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



Even in this case, similar to what we observed with Word2Vec, the results show a reasonable separation of word clusters based on the given centers.  

However, a few unexpected or seemingly unrelated words still appear within the same group. It's interesting notice that most of these outliers are found in the cluster associated with the center word "dog", which was not present in the training vocabulary.  
This highlights one of the key strengths of FastText: its ability to generate embeddings for out-of-vocabulary (OOV) words by leveraging subword information.  
As a result, FastText proves more robust and suitable for handling unseen or rare terms.

### Description Embedding

The main goal of this section is to effectively convert a description into a vector representation using FastText.
To achieve this, we calculate the mean or the sum of the embeddings of each token in the description, which results in the final description embedding.

In [ ]:
#Convert a description to a vector using FastText.
def description_to_vector(description, model, useSum=True):
    # Separating sentences
    description= re.sub(r'\n', ' ', description)
    description_sentences = re.split(r'[.!?]\s', description)

    # Tokenize and process each sentence
    description_tokens_vectorized = [] # Will contain the vectors of the tokens
    for sent in description_sentences:
        if len(sent) > 0:
            sentence_tokens = custom_tokenize(sent) # Tokenize the sentence
            for token in sentence_tokens:
                if token in model.wv.key_to_index:
                    description_tokens_vectorized.append(model.wv[token])
                else:# If the token is not in the vocabulary, use the FastText model to get its vector
                    # This is useful for out-of-vocabulary words
                    description_tokens_vectorized.append(model.wv.get_vector(token))
    
    if len(description_tokens_vectorized) > 0:
        # If useSum is True, sum the vectors of the tokens
        # Otherwise, return the mean vector
        if useSum:
            return np.sum(description_tokens_vectorized, axis=0)
        else:
            return np.mean(description_tokens_vectorized, axis=0)
    else:
        return np.zeros(model.vector_size)

    

We create a new column in the Dataset containing the problems assigning for each one the corresponding description-vector.   
We use the previous function description_to_vector with useSum=True (we are summing the embedding of each token)

In [ ]:
# Make sure the column 'description_vector' exists
df_problem_and_human_solutions_list_we['description_vector'] = None


# Populate the column with the vectors
for i, row in df_problem_and_human_solutions_list_we.iterrows():
    description = row['description']
    if isinstance(description, str):
        # use the sum of the vectors as description vector
        vector = description_to_vector(description, ft_model, useSum=True)
    else:
        vector = np.zeros(ft_model.get_dimension())

    # Assign the vector to the corresponding cell in the dataframe
    df_problem_and_human_solutions_list_we.at[i, 'description_vector'] = vector

Just to check the shape of the vectors

In [ ]:
df_problem_and_human_solutions_list_we["description_vector"]

Extracting tags from codeforces_specific_metadata and adding them to a specific column 'cf_tags' of the dataframe.

In [ ]:
df_problem_and_human_solutions_list_we['cf_tags'] = df_problem_and_human_solutions_list_we['codeforces_specific_metadata'].apply(lambda x: x.get('cf_tags') if isinstance(x, dict) else None)

This step is designed for memory efficiency and reproducibility.
Since computing description vectors is resource-intensive and can lead to out-of-memory (OOM) errors on large datasets, we store the precomputed vectors along with the associated tags in a reduced DataFrame.  
We then serialize it using Pickle, allowing for fast reloading in future sessions without the need to reprocess the text.

In [ ]:
# Save the description vectors and tags to a pickle file
df_reduced = df_problem_and_human_solutions_list_we[["description_vector", "cf_tags"]].copy()
df_reduced.to_pickle("description_vectors_and_tags.pkl")

In [ ]:
# Load the reduced DataFrame from a pickle file (if already saved)
# This avoids recalculating vectors every time the notebook runs
import pandas as pd
df_reduced = pd.read_pickle("description_vectors_and_tags.pkl")

We can see some of the tags only to have an idea.

In [24]:
df_reduced['cf_tags']

0       [constructive algorithms, dfs and similar, dp,...
1       [data structures, dp, implementation, shortest...
2               [constructive algorithms, implementation]
3                                                [greedy]
4       [constructive algorithms, data structures, dp,...
                              ...                        
3100                                                   []
3101                                                   []
3102                                                   []
3103                                                   []
3104                                                   []
Name: cf_tags, Length: 3105, dtype: object

Reintegrate the saved vectors and tags into the main DataFrame

In [ ]:
#Insert the tags and the description vector in the dataframe
df_problem_and_human_solutions_list_we["cf_tags"]= df_reduced['cf_tags']
df_problem_and_human_solutions_list_we["description_vector"]= df_reduced['description_vector']
# Quick inspection of the updated DataFrame to confirm integration
df_problem_and_human_solutions_list_we.head()

,problem_id,problem_name,description,correct_solution_list,data_source,source_specific_limits,codeforces_specific_metadata,tests,human_accuracy_rate,dataclass,complexity_framework,description_vector,cf_tags
0,0,339_C. Xenia and Weights,Xenia has a set of weights and pan scales. Eac...,"[{'solution_id': '0_0', 'solution_code': '__au...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 339, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '0000000101 3 ', '...",0.281633,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.40579710144927...,"[40.88922, 68.51662, 6.8400073, -3.7496269, 10...","[constructive algorithms, dfs and similar, dp,..."
1,1,1547_E. Air Conditioners,On a strip of land of length n there are k air...,"[{'solution_id': '1_0', 'solution_code': 'def ...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 1547, 'cf_index': 'E', 'cf_p...",{'public_tests': [{'input': '5 6 2 2 5 14 16 ...,0.620833,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.11409395973154...,"[-10.14601, 193.53925, 24.337927, 79.53583, 23...","[data structures, dp, implementation, shortest..."
2,2,268_C. Beautiful Sets of Points,Manao has invented a new mathematical term — a...,"[{'solution_id': '2_0', 'solution_code': 'if _...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 268, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '2 2 ', 'output': ...",0.525066,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.05527638190954...,"[-6.0463147, 75.508385, 18.788363, 13.632879, ...","[constructive algorithms, implementation]"
3,3,478_C. Table Decorations,"You have r red, g green and b blue balloons. T...","[{'solution_id': '3_0', 'solution_code': 'a = ...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 478, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '1 1 1 ', 'output'...",0.562264,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.41610738255033...,"[5.330355, 36.27579, 7.763661, 2.4407022, 53.7...",[greedy]
4,4,5_C. Longest Regular Bracket Sequence,This is yet another problem dealing with regul...,"[{'solution_id': '4_0', 'solution_code': 'stri...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 5, 'cf_index': 'C', 'cf_poin...","{'public_tests': [{'input': ')((())))(()()) ',...",0.395939,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.03846153846153...,"[-20.027811, 24.79554, 7.7614923, 13.442029, 3...","[constructive algorithms, data structures, dp,..."


In order to explore a potential correlation between the description embeddings and the cf_tags, this procedure randomly selects a portion of the Dataframe with specific values of cf_tags (selected_tags).    

We know that each problem can have multiple tags, sometimes even correlated one with each other.  
For this reason, we isolate problems that belong only to one of the given tags (e.g., "greedy" but not "brute force" or "binary search"), in this way we try to reduce confounding effects due to overlapping concepts and allow for clearer, tag-specific embedding analysis.

In [ ]:
# Select the problems with specific tags to analyze

selected_tags = ["greedy", "brute force", "binary search"]

# List to store the samples
samples = []

for tag in selected_tags:
    other_tags = [t for t in selected_tags if t != tag]

    # Filter the DataFrame for the specific tag and not any of the other tags
    df_tag = df_problem_and_human_solutions_list_we[df_problem_and_human_solutions_list_we["cf_tags"].apply(
            lambda tags: tag in tags and not any(t in tags for t in other_tags)
        )]
    df_sample = df_tag.sample(n=20, random_state=42)  # Select 20 samples for each tag
    df_sample = df_sample.copy()
    df_sample["selected_tag"] = tag  # Assign the selected tag to the sample
    samples.append(df_sample)

# Concatenate all samples into a single DataFrame
df_selected = pd.concat(samples, ignore_index=True)
# See if it works
df_selected

,problem_id,problem_name,description,correct_solution_list,data_source,source_specific_limits,codeforces_specific_metadata,tests,human_accuracy_rate,dataclass,complexity_framework,description_vector,cf_tags,selected_tag
0,1356,1167_D. Bicolored RBS,A string is called bracket sequence if it does...,"[{'solution_id': '1356_0', 'solution_code': '#...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 1167, 'cf_index': 'D', 'cf_p...","{'public_tests': [{'input': '2 () ', 'output':...",0.690355,{'dataclass_code': 'import sys import time imp...,"{'time_complexity_fail_rate': 1.0, 'space_comp...","[-4.0829473, 89.61917, 10.330913, 26.139729, 1...","[constructive algorithms, greedy]",greedy
1,2822,276_D. Little Girl and Maximum XOR,A little girl loves problems on bitwise operat...,"[{'solution_id': '2822_0', 'solution_code': 'd...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 276, 'cf_index': 'D', 'cf_po...","{'public_tests': [{'input': '8 16 ', 'output':...",0.581081,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.84883720930232...,"[-2.0882452, 34.08194, 10.20857, 11.062459, 64...","[bitmasks, dp, greedy, implementation, math]",greedy
2,2989,1515_A. Phoenix and Gold,"Phoenix has collected n pieces of gold, and he...","[{'solution_id': '2989_0', 'solution_code': 'i...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 1515, 'cf_index': 'A', 'cf_p...",{'public_tests': [{'input': '3 3 2 3 2 1 5 3 1...,0.500000,{'dataclass_code': 'import sys import time imp...,"{'time_complexity_fail_rate': 1.0, 'space_comp...","[38.512028, 87.194534, 15.281353, 18.964302, 1...","[constructive algorithms, greedy, math]",greedy
3,2545,1416_C. XOR Inverse,You are given an array a consisting of n non-n...,"[{'solution_id': '2545_0', 'solution_code': 'n...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 1416, 'cf_index': 'C', 'cf_p...","{'public_tests': [{'input': '3 8 10 3 ', 'outp...",0.688312,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.13207547169811...,"[-1.4355078, 80.56184, 1.9877968, 30.703989, 9...","[bitmasks, data structures, divide and conquer...",greedy
4,1960,1311_A. Add Odd or Subtract Even,You are given two positive integers a and b.\n...,"[{'solution_id': '1960_0', 'solution_code': 'f...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 1311, 'cf_index': 'A', 'cf_p...",{'public_tests': [{'input': '5 2 3 10 10 2 4 7...,0.644186,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.01805054151624...,"[14.694276, 63.40165, 9.389306, 5.422409, 75.3...","[greedy, implementation, math]",greedy
5,2454,955_A. Feed the cat,"After waking up at hh:mm, Andrew realised that...","[{'solution_id': '2454_0', 'solution_code': 'h...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 955, 'cf_index': 'A', 'cf_po...",{'public_tests': [{'input': '19 00 255 1 100 1...,0.584577,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.05106382978723...,"[18.31242, 77.77756, 19.792704, 7.910349, 103....","[greedy, math]",greedy
6,2242,758_D. Ability To Convert,Alexander is learning how to convert numbers f...,"[{'solution_id': '2242_0', 'solution_code': 'n...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 758, 'cf_index': 'D', 'cf_po...","{'public_tests': [{'input': '16 11311 ', 'outp...",0.448276,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.11538461538461...,"[-13.140537, 56.548748, 9.355681, 21.291874, 5...","[constructive algorithms, dp, greedy, math, st...",greedy
7,476,1442_B. Identify the Operations,"We start with a permutation a_1, a_2, …, a_n a...","[{'solution_id': '476_0', 'solution_code': 'mx...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf

#### Visualizing Description Embeddings with t-SNE

Plot the 3D projection of the description vectors, with colors representing the selected_tag, to observe whether there are any patterns or cluster related to the corresponding cf_tag.   

In [ ]:
from sklearn.manifold import TSNE
import numpy as np

vectors = np.array(df_selected["description_vector"].tolist())

tsne = TSNE(n_components=3, random_state=42, perplexity=10)
vectors_3d = tsne.fit_transform(vectors)

df_selected[["x", "y", "z"]] = vectors_3d

import plotly.express as px

fig = px.scatter_3d(
    df_selected,
    x="x", y="y", z="z",
    color="selected_tag",
    text="problem_name",
    hover_data=["problem_name", "selected_tag"],
    title="3D Projection of Descriptions by Tag"
)

fig.update_traces(marker=dict(size=5, opacity=0.8))
fig.show()

The previous plot does not show a clear distinct separation based on cf_tags, at least within the explored three-dimensional space using TSNE.

This observation is realistic and expected for several reasons:
- t-SNE is non-linear and local: it preserves neighborhood structures, but not global separation.
- FastText embeddings are unsupervised: the model was not trained with tag supervision, so there's no guarantee that tag information is encoded linearly or separably.
- Descriptions may share semantics across tags: for example, both “greedy” and “brute force” problems may use similar language like loops or conditions.

### Solution Handling

The main goal of this section is to effectively convert a solution code into a vector representation using FastText.
To achieve this, we calculate the mean or the sum of the embeddings of each token of the solution, as we did for the description embedding part.

To do this we:
- Tokenize each solution using a default python tokenizer (the solutions are in python)

- Train FastText to extract the word embeddings based on the context of each solution

- Explore different way of managing comments in the solution code 

- Aggregate the word embeddings in a single solution representation (solution embedding)

#### Tokenization and Word Embeddings

Extracting the solution codes from the 'correct_solution_list' column and showing them

In [105]:
all_solutions = []

def extract_codes(solution_list):
    if isinstance(solution_list, list):
        return [item['solution_code'] for item in solution_list
                if isinstance(item, dict) and isinstance(item.get('solution_code'), str)]
    return []

df_problem_and_human_solutions_list_we['correct_solution_list'].apply(
    lambda x: all_solutions.extend(extract_codes(x))
)

all_solutions[0:10]

['__author__ = \'ratnesh.mishra\'\n\nweights = map(int, input())\n\nweights = [cnt for cnt, x in enumerate(weights, 1) if x]\n\nm = int(input())\n\nstate = [(0, 0, 0, [])]\n\nres = "NO"\nwhile state:\n    w, b, k, l = state.pop()\n\n    if k == m:\n        res = \'YES\\n\' + \' \'.join(map(str, l))\n        break\n\n    for wt in weights:\n        if wt != w and wt > b:\n            state.append((wt, wt-b, k+1, l+[wt]))\n\nprint(res)\n\n',
 "s = input()\nm = int(input())\nk1 = k2 = 0\nz = [0]*1001; index = 0\nx = True\np = list(s)\nif p.count('1') == 3:\n    i = 0\n    while p[i] == '0':\n        i += 1\n    i += 1\n    while p[i] == '0':\n        i += 1\n    z[1] = i+1\n    k2 = z[1]\n    p = 1\n    index = 1\nelse: p = 0\nfor j in range(p, m):\n    for i in range(10):\n        if s[i] != '0' and i+1 != z[index] and k1 + (i+1) > k2:\n            k1 += (i+1)\n            index += 1\n            z[index] = i+1;\n            break\n    else:\n        x = False\n        break\n    if inde

This function tokenizes Python code using the built-in tokenize module.  
It removes in-line comments and empty tokens to keep the meaningful code structure. 

It receive the string with the code and return the list of tokens associated.

In [ ]:
import io
import tokenize # Python standard library tokenizer
from typing import List

def tokenize_code_only(code: str) -> List[str]:
    tokens = []
    try:
        token_generator = tokenize.generate_tokens(io.StringIO(code).readline)
        # tokenum is the type of token, tokval is the value
        for toknum, tokval, _, _, _ in token_generator: 
            if toknum != tokenize.COMMENT and tokval.strip() != "":
                tokens.append(tokval)
    except tokenize.TokenError:
        pass
    return tokens

This function instead is an alternative of the previous one and is used to tokenize the code without removing comments and empty lines.

In [ ]:
def tokenize_code(code: str) -> List[str]:
    tokens = []
    try:
        token_generator = tokenize.generate_tokens(io.StringIO(code).readline)
        # tokenum is the type of token, tokval is the value
        for toknum, tokval, _, _, _ in token_generator:
            if tokval.strip() != "":
                tokens.append(tokval)
    except tokenize.TokenError:
        pass  # not valid code, skip it
    return tokens


Remove multi-line comments (triple-quoted strings) from Python code.
- Multi-line comments: not considered in the previous function but present in our data. 

In [ ]:
import re
def remove_multiline_comments(code: str) -> str:

    # Pattern to match triple-quoted strings: '''...''' or """..."""
    # This is a basic pattern that assumes such strings are used only for comments
    multiline_comment_pattern = r"(\'\'\'[\s\S]*?\'\'\'|\"\"\"[\s\S]*?\"\"\")"

    # Remove also very long words
    cleaned_code = re.sub(r"\b\w{21,}\b", "", code)

    # Replace all matches with an empty string
    cleaned_code = re.sub(multiline_comment_pattern, '', cleaned_code)

    return cleaned_code

Example code snippets to test the function

In [ ]:
examples_code = ["""
def example_function(x, y):
    # This function adds two numbers
    result = x + y  # Add x and y
    return result  # Return the result
""", """
def another_function(a, b): jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj
    # This function multiplies two numbers
    product = a * b  # Multiply a and b
    return product  # Return the product
""", """
for i in range(10):  # stampa i
    print(i)
str = '''Stringa assegnata e valida'''
# Commento
'''
Questo è un commento multilinea non assegnato
'''
""",
"def fdjjnf another ssst_: '''\n111111101000101111000110101011010\n100000100111010101111100100011011\n101110101001010000101000111111000\n101110100011010010010111111011010\n101110100100011011110110101110000\n100000100110011001111100111100000\n111111101101000101001101110010001?''' \n # This function does something \n return 42",
'def example_function(x, y):\n """ This function adds two numbers""" '
]

Let's remove the multiline and inline comments and tokenize the example data

In [77]:
clened_code_ex = [remove_multiline_comments(code) for code in examples_code]

tokenized_code_ex = [tokenize_code_only(code) for code in clened_code_ex if len(code.strip()) > 0]

In [ ]:
print("Number of tokenized code solutions:", len(tokenized_code_ex))
for i, code in enumerate(tokenized_code_ex):
    if i >= 3:  # Limit to first 3 codes for brevity
        break
    print(f"Code {i+1}:{examples_code[i]}\nTokens: {code}\n")

Number of tokenized code solutions: 5
Code 1:
def example_function(x, y):
    # This function adds two numbers
    result = x + y  # Add x and y
    return result  # Return the result 

Tokens: ['def', 'example_function', '(', 'x', ',', 'y', ')', ':', 'result', '=', 'x', '+', 'y', 'return', 'result']

Code 2:
def another_function(a, b): jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj
    # This function multiplies two numbers
    product = a * b  # Multiply a and b
    return product  # Return the product

Tokens: ['def', 'another_function', '(', 'a', ',', 'b', ')', ':', 'product', '=', 'a', '*', 'b', 'return', 'product']



Tokenize the real code and remove comments and empty lines  
(to save time it's also possible to load directly the tokenized code from a saved file as shown in the following cells)

In [ ]:
clened_code = [remove_multiline_comments(code) for code in all_solutions]

tokenized_code = [tokenize_code_only(code) for code in clened_code if len(code.strip()) > 0]

In [2]:
import pickle

In [ ]:
# Store the tokenized code in a pickle file for easy access later
with open("tokenized_code.pkl", "wb") as f:
    for item in tokenized_code:
        pickle.dump(item, f)

In [ ]:
# Load the tokenized code from the pickle file
tokenized_code = []

with open("tokenized_code.pkl", "rb") as f:
    while True:
        try:
            item = pickle.load(f)
            tokenized_code.append(item)
        except EOFError:
            break

Print the number of tokenized code solutions and the first 3 tokenized codes

In [ ]:
print("Number of tokenized code solutions:", len(tokenized_code))
for i, code in enumerate(tokenized_code):
    if i > 2:  # Limit to first 3 codes for brevity
        break
    print(f"Code {i+1}:\n Tokens: {code}")

Number of tokenized code solutions: 1190250
Code 1:
 Tokens: ['__author__', '=', "'ratnesh.mishra'", 'weights', '=', 'map', '(', 'int', ',', 'input', '(', ')', ')', 'weights', '=', '[', 'cnt', 'for', 'cnt', ',', 'x', 'in', 'enumerate', '(', 'weights', ',', '1', ')', 'if', 'x', ']', 'm', '=', 'int', '(', 'input', '(', ')', ')', 'state', '=', '[', '(', '0', ',', '0', ',', '0', ',', '[', ']', ')', ']', 'res', '=', '"NO"', 'while', 'state', ':', 'w', ',', 'b', ',', 'k', ',', 'l', '=', 'state', '.', 'pop', '(', ')', 'if', 'k', '==', 'm', ':', 'res', '=', "'YES\\n'", '+', "' '", '.', 'join', '(', 'map', '(', 'str', ',', 'l', ')', ')', 'break', 'for', 'wt', 'in', 'weights', ':', 'if', 'wt', '!=', 'w', 'and', 'wt', '>', 'b', ':', 'state', '.', 'append', '(', '(', 'wt', ',', 'wt', '-', 'b', ',', 'k', '+', '1', ',', 'l', '+', '[', 'wt', ']', ')', ')', 'print', '(', 'res', ')']
Code 2:
 Tokens: ['s', '=', 'input', '(', ')', 'm', '=', 'int', '(', 'input', '(', ')', ')', 'k1', '=', 'k2', '=', '0', 

Train the FastText model on the previously tokenized code corpus.  
We are using almost the same configuration we had for descriptions.

In [ ]:
# Create FastText model for code solutions
from gensim.models import FastText
ft_model_code = FastText(
    vector_size=100,
    window=5,
    min_count=2,
    workers=8,
    sg=1
)
ft_model_code.build_vocab(corpus_iterable=tokenized_code)
# Train the model on the tokenized code solutions
ft_model_code.train(
    corpus_iterable=tokenized_code,
    total_examples=len(tokenized_code),
    epochs=10
)
# Save the FastText model for code solutions
ft_model_code.save("fasttext_code.model")

Load the model, useful to skip the previous long training step

In [118]:
ft_model_code = FastText.load("fasttext_code.model")

In [ ]:
# Print the vocabulary size
print("Vocabulary size:", len(ft_model_code.wv.key_to_index))

Vocabulary size: 266766


Here, test whether the model can recognize the most similar words to some key formal python terms (e.g., "for", "if", "while", etc.).  
This gives us insight into whether the model has learned meaningful relationships between programming concepts based on the tokenized code.

In [120]:
ft_model_code.wv.most_similar('for', topn=10)

[('in', 0.8137962222099304),
 ('range', 0.7991275787353516),
 ('i', 0.7658703327178955),
 ('if', 0.7493084669113159),
 ('n', 0.7363026142120361),
 ('1', 0.7334885001182556),
 ('int', 0.7325080037117004),
 ('[', 0.7216368913650513),
 (')', 0.7215696573257446),
 (']', 0.7209526300430298)]

In [121]:
ft_model_code.wv.most_similar('if', topn=10)

[(':', 0.8620245456695557),
 ('1', 0.859686553478241),
 ('0', 0.8515041470527649),
 ('i', 0.8422074317932129),
 (')', 0.834545910358429),
 ('==', 0.8290315866470337),
 ('-', 0.817014217376709),
 ('(', 0.8148909211158752),
 ('+', 0.8006325960159302),
 ('n', 0.7996149659156799)]

In [122]:
ft_model_code.wv.most_similar('while', topn=10)

[('whiled', 0.787043035030365),
 ('n_while', 0.649196982383728),
 ('-=', 0.6250849366188049),
 ('no_while', 0.6032726168632507),
 ('<=', 0.5898699760437012),
 ('whileloop_count', 0.5862532258033752),
 ('continue_while', 0.583503246307373),
 ('elif', 0.5788335204124451),
 ('!=', 0.5729063153266907),
 ('>=', 0.5720224380493164)]

In [123]:
ft_model_code.wv.most_similar('import', topn=10)

[('from', 0.8952745795249939),
 ('importlib', 0.8554027676582336),
 ('impor', 0.8284448385238647),
 ('collections', 0.8251305222511292),
 ('itertools', 0.791632890701294),
 ('bisect', 0.7599672675132751),
 ('_collections', 0.7570397853851318),
 ('functools', 0.7509666085243225),
 ('as', 0.7279273271560669),
 ('fractions', 0.7246843576431274)]

In [124]:
ft_model_code.wv.most_similar('[', topn=10)

[(']', 0.9672830104827881),
 ('i', 0.8931779861450195),
 ('1', 0.8762338161468506),
 ('-', 0.8599721193313599),
 ('in', 0.8263137340545654),
 ('=', 0.8251293301582336),
 ('0', 0.8235974907875061),
 (')', 0.822938859462738),
 (':', 0.8216568827629089),
 ('(', 0.8127009272575378)]

It seems to have captured some structures and repetition in the code language 

Let's try to see a graphical representation of some random tokens in the vocabulary using t-SNE and plotly.

In [125]:
import random

sample = random.sample(list(ft_model_code.wv.key_to_index), 50)
print(sample)

['5242715496', 'duplicate', 'x_dis', '13723', 'aLngth', 'listout', 'kxb', 'str_R', '45697', 'check_pythagorean_theorem', '6379198553380056131398567110588', '194164134', 'cal_arr', 'inpb', 'one_sec', 'black_distance', '69119', '12301', 'pref_sm', 'numberOfEven', 'odd_half', '72497', 'indLeft', 'money_2', '140.0', 'max_of_list', 'curr_side', 'index_min_r', '44447447774', 'lenFlag', '203734831851444523998150720000', '77167168', '3211915413467943245280408135253628171660543181148276654080000', '32377', 'par_u', 'allowed_letters', 'ans_res', '" YNeos"', 'all_safes', 'principio', '173674546470380440741680', 'berlist', "'f(24) = MAGNA NIMIS!'", 'col_taken', '11876492948655423122013963214755495466938777600', '227705360181788283783595008', 'tiempo1estufa', 'ans_cand', '178596353224677595625526937762950163660800', 'minitem']


In [ ]:
# Qbtain the word vectors for the sampled words
word_vectors = ft_model_code.wv[sample]
print(word_vectors)
print("Shape of word vectors:", word_vectors.shape)

[[-0.17741325 -0.40698034  0.19850037 ... -0.21567681  1.6538343
  -0.27807835]
 [ 0.49476406 -0.633559   -0.13349868 ...  0.96943897  0.6510257
   0.5175061 ]
 [ 0.6048307   0.5656388   0.6332588  ...  1.2473801  -0.4507485
   0.6731845 ]
 ...
 [-0.12977535  0.06597771 -0.35987705 ... -0.124568    0.2892887
   0.24415283]
 [-0.15097976 -0.45376438  0.55178446 ... -0.42563334  1.6015333
  -0.9339056 ]
 [ 0.652093    0.12511691 -0.07387293 ...  1.0632232   0.2472515
   0.4658623 ]]
Shape of word vectors: (50, 100)


In [ ]:
from sklearn.manifold import TSNE
import numpy as np

# Perform t-SNE to reduce the dimensionality of the word vectors to 3D
tsne = TSNE(n_components=3, n_iter=2000)
tsne_embedding = tsne.fit_transform(word_vectors)

x, y, z = np.transpose(tsne_embedding)

import plotly.express as px

fig = px.scatter_3d(x=x[:200],y=y[:200],z=z[:200],text=sample[:200])
fig.update_traces(marker=dict(size=3,line=dict(width=2)),textfont_size=10)
fig.show()

c:\Users\utente\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



Not really informative.

#### Labeling Comments

The main purpose is to train a FastText model using as tokens also the words of the in-line comments with a special tag added to them.    
This could allow the model to better distinguish comments from pure code.

Here, the new version of the tokenize function (marks comments with a prefix).


In [ ]:
import io
import tokenize
from typing import List

def tokenize_code_with_comments(code: str) -> List[str]:
    tokens = []
    try:
        token_generator = tokenize.generate_tokens(io.StringIO(code).readline)
        for toknum, tokval, _, _, _ in token_generator:
            tokval = tokval.strip()
            if tokval == "":
                continue

            if toknum == tokenize.COMMENT:
                # Remove "#" and tokenize comment
                comment_body = tokval.lstrip("#").strip()
                comment_tokens = comment_body.split()
                for word in comment_tokens:
                    tokens.append(f"<comment>{word.lower()}")
            else:
                tokens.append(tokval)
    except (tokenize.TokenError, IndentationError, SyntaxError) as e:
        # Skip bad code
        pass
    except Exception as e:
        pass
    return tokens


Tokenize examples of code maintaining in-line comments with an additional tag.

In [ ]:

cleaned_code_ex = [remove_multiline_comments(code) for code in examples_code]
tokenized_code_with_comments_ex = [tokenize_code_with_comments(code) for code in cleaned_code_ex if len(code.strip()) > 0]

We try this new type of tokenization on the previous example of snippet (used to test the tokenization without comments)

In [ ]:

print("Number of tokenized code solutions:", len(tokenized_code_with_comments_ex))
for i, code in enumerate(tokenized_code_with_comments_ex):
    if i >= 3:  # Limit to first 3 codes for brevity
        break
    print(f"Code {i+1}:\n Tokens: {code}")

Number of tokenized code solutions: 5
Code 1:
 Tokens: ['def', 'example_function', '(', 'x', ',', 'y', ')', ':', '<comment>this', '<comment>function', '<comment>adds', '<comment>two', '<comment>numbers', 'result', '=', 'x', '+', 'y', '<comment>add', '<comment>x', '<comment>and', '<comment>y', 'return', 'result', '<comment>return', '<comment>the', '<comment>result']
Code 2:
 Tokens: ['def', 'another_function', '(', 'a', ',', 'b', ')', ':', '<comment>this', '<comment>function', '<comment>multiplies', '<comment>two', '<comment>numbers', 'product', '=', 'a', '*', 'b', '<comment>multiply', '<comment>a', '<comment>and', '<comment>b', 'return', 'product', '<comment>return', '<comment>the', '<comment>product']
Code 3:
 Tokens: ['for', 'i', 'in', 'range', '(', '10', ')', ':', '<comment>stampa', '<comment>i', 'print', '(', 'i', ')', 'str', '=', '<comment>commento']


Tokenize now the real code

In [110]:
cleaned_code= [remove_multiline_comments(code) for code in all_solutions]
tokenized_code_with_comments= [tokenize_code_with_comments(code) for code in cleaned_code  if len(code.strip()) > 0]

Here the usual saving/load options in order to save time for the tokenization

In [ ]:
# Optional: Save the tokenized code with comments to a file
with open("tokenized_code_wcomm.pkl", "wb") as f:
    for item in tokenized_code_with_comments:
        pickle.dump(item, f)

In [ ]:
# Load the tokenized code with comments from the file
tokenized_code_with_comments = []

with open("tokenized_code_wcomm.pkl", "rb") as f:
    while True:
        try:
            item = pickle.load(f)
            tokenized_code_with_comments.append(item)
        except EOFError:
            break


Print the real number of tokenized code solutions and a few examples to have an idea

In [ ]:
print("Number of tokenized code solutions:", len(tokenized_code_with_comments))
for i, code in enumerate(tokenized_code_with_comments):
    if i >= 5:  # Limit to first 5 codes for brevity
        break
    print(f"Code {i+1}:\n Tokens: {code}")

Number of tokenized code solutions: 1190250
Code 1:
 Tokens: ['__author__', '=', "'ratnesh.mishra'", 'weights', '=', 'map', '(', 'int', ',', 'input', '(', ')', ')', 'weights', '=', '[', 'cnt', 'for', 'cnt', ',', 'x', 'in', 'enumerate', '(', 'weights', ',', '1', ')', 'if', 'x', ']', 'm', '=', 'int', '(', 'input', '(', ')', ')', 'state', '=', '[', '(', '0', ',', '0', ',', '0', ',', '[', ']', ')', ']', 'res', '=', '"NO"', 'while', 'state', ':', 'w', ',', 'b', ',', 'k', ',', 'l', '=', 'state', '.', 'pop', '(', ')', 'if', 'k', '==', 'm', ':', 'res', '=', "'YES\\n'", '+', "' '", '.', 'join', '(', 'map', '(', 'str', ',', 'l', ')', ')', 'break', 'for', 'wt', 'in', 'weights', ':', 'if', 'wt', '!=', 'w', 'and', 'wt', '>', 'b', ':', 'state', '.', 'append', '(', '(', 'wt', ',', 'wt', '-', 'b', ',', 'k', '+', '1', ',', 'l', '+', '[', 'wt', ']', ')', ')', 'print', '(', 'res', ')']
Code 2:
 Tokens: ['s', '=', 'input', '(', ')', 'm', '=', 'int', '(', 'input', '(', ')', ')', 'k1', '=', 'k2', '=', '0', 

Train the FastText model on the new tokenized corpus containing labeled in-line comments

In [ ]:
# Train FastText model on the tokenized code with comments
from gensim.models import FastText
ft_model_code = FastText(
    vector_size=100,
    window=5,
    min_count=2,
    workers=8,
    sg=1
)
ft_model_code.build_vocab(corpus_iterable=tokenized_code_with_comments)
ft_model_code.train(
    corpus_iterable=tokenized_code_with_comments,
    total_examples=len(tokenized_code_with_comments),
    epochs=10
)
# Save the FastText model for code solutions with comments
ft_model_code.save("fasttext_code_with_comments.model")

Load option if you want to skip long training

In [57]:
ft_model_code = FastText.load("fasttext_code_with_comments.model")

In [58]:
print("Vocabulary size:", len(ft_model_code.wv.key_to_index))

Vocabulary size: 336615


We can notice a bigger vocabulary than the previous FastText model due to the extension of the training data we did including comments.

In [135]:
ft_model_code.wv.most_similar('for', topn=10)

[('range', 0.8057198524475098),
 ('in', 0.802432119846344),
 ('i', 0.7367343306541443),
 ('input', 0.7059527039527893),
 ('n', 0.7015922665596008),
 ('if', 0.6915501952171326),
 ('j', 0.6869797706604004),
 ('1', 0.6843733787536621),
 ('len', 0.6766437292098999),
 ('int', 0.6735730767250061)]

In [137]:
ft_model_code.wv.most_similar('if', topn=10)

[(':', 0.8679385781288147),
 ('1', 0.8470319509506226),
 ('0', 0.8419544696807861),
 ('i', 0.8319261074066162),
 (')', 0.8250016570091248),
 ('==', 0.8208984732627869),
 ('(', 0.8130518198013306),
 ('=', 0.8098618984222412),
 ('-', 0.8023994565010071),
 ('+', 0.78610759973526)]

In [ ]:
ft_model_code.wv.most_similar('comment', topn=10)

[('comments', 0.9467008709907532),
 ('commentslist', 0.9408623576164246),
 ('commentlist', 0.9154900312423706),
 ('<comment>#uncomment', 0.90512615442276),
 ('<comment>ちょうどいい位置(ここより手前で2**iまでを網羅)にバイパスを作る感じ', 0.871245801448822),
 ('<comment>g[cur]:', 0.8651385307312012),
 ('<comment>comment', 0.861804187297821),
 ('<comment>读取第一行的n', 0.8603671193122864),
 ('comment_result', 0.8591660857200623),
 ('<comment>完全に被覆している区間は狭い方だけ残す', 0.8565295934677124)]

We can see that there are many comments in other strange languages that are not useful.  
Anyway the programming words seems to have maintained their top neighbors.

Let's try to visualize again the obtained word embeddings on some random tokens

In [ ]:
import random
# Sample 100 random words from the vocabulary
sample = random.sample(list(ft_model_code.wv.key_to_index), 100)
print(sample)

['<comment>get_primes(n):', 'ddl', '<comment>f(n//2,', 'g1c', 'increas_time', 'hq9_lang', 'start_previous', 'pidorskie_zadachi', '"backward"', '2419', 'voyageCount', 'te2', '<comment>sallii', 'list_twisted', '103043307', 'triangleNum', 'num_icecream', 'matline', 'biggest_left', 'twoSubs', 'isValidName', '59899999999', '<comment>lst1.append(lst[i])', '8314', 'loads', 'can_sort', '6606028800', 'list_string1', '<comment>converting', '<comment>print(lst)', '306867765', 'sfor1', 'has_amt', 'b_a', '<comment>print(sum(value[:k]))', 'evenIdx', 'minimR', 'phonenumnum', '<comment>print(dup_pse_list)', 'max_ob', 'nas', '101001110', '<comment>found.append(i)', 'calculatefunction2', 'biggestAm', 'matrix_check_1', '<comment>ans[i],', 'timea1', '530134554', 'part_two', 'ListOne', '<comment>2*count)', 'xab', 'getVs', '19316', '<comment>print(s[:i+1])', 'kisibasL', 'node_list2', 'skier', '44447777', 'maxRange', '81071', 'opens', 'STACK', 'allPerm', 'males', '<comment>â', '762491092', 'kx1', '" % d"', '

In [147]:
# Get the word vectors for the sampled words
word_vectors = ft_model_code.wv[sample]
print(word_vectors)
print("Shape of word vectors:", word_vectors.shape)

[[ 0.88286495  1.2400173   0.17885505 ... -1.1880517   0.2806537
  -0.21235675]
 [ 1.0490906   0.14483853  0.37600923 ... -1.1213222   0.7860192
  -1.0733389 ]
 [ 0.30698267  1.4658773   0.04503308 ... -0.3770461  -0.03734222
  -0.09637897]
 ...
 [ 0.32622108 -0.47709632 -0.30316356 ... -0.65952617  0.06701706
  -0.94742084]
 [ 0.32318535  0.52291054 -0.39157492 ... -1.1882232  -0.2062516
   0.15264748]
 [ 0.1740282  -0.50651467  0.2728058  ... -0.39571142 -0.14412765
   0.7267321 ]]
Shape of word vectors: (100, 100)


Using again T-SNE

In [ ]:
from sklearn.manifold import TSNE
import numpy as np

tsne = TSNE(n_components=3, n_iter=2000)
tsne_embedding = tsne.fit_transform(word_vectors)

x, y, z = np.transpose(tsne_embedding)

import plotly.express as px

fig = px.scatter_3d(x=x[:200],y=y[:200],z=z[:200],text=sample[:200])
fig.update_traces(marker=dict(size=3,line=dict(width=2)),textfont_size=10)
fig.show()

c:\Users\utente\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



The obtained visualization seems to be not really informative, containing many nonsensical terms (especially in the tokens of comments):
- including rare variable names, symbols, or malformed code. 

Beyond this, the model seems to have identified some groups such as: one containing comments and another one containing numbers. 


Now let's try to analyze a more structured visualization plotting the top most similar embeddings for some of domain-specific words. We will use the usual function of the previous sections.

In [ ]:
# Example 
centers=["for", "if", "comment", "print", "dog"]
plot_word_embedding_grouped(ft_model_code, centers)

c:\Users\utente\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



We observe a separation between the comment-tagged tokens and the other code-symbols also in this plot. 

Additionally, there is a noticeable distinction between the word 'dog' (which is not even in the vocabulary) and the others, highlighting their semantic distance.

#### Solution Embeddings

The main goal is to convert a python solution (code) into a single vector by aggregating all its token embeddings   
(obtained with the tokenize function with comments we have seen before).


The following function has exactly this purpose (convert code to vector)  
Parameters:
- code (str): The code string.
- ft_model: The trained FastText model.
- use_sum (bool): If True, sum embeddings; if False, compute the mean.

Return:
- np.ndarray: The aggregated vector representing the code.

In [ ]:
from typing import List
import numpy as np
def solution_to_vector(code: str, ft_model, use_sum: bool = False) -> np.ndarray:
    tokens = tokenize_code_with_comments(code)  # using the tokenization function with comments

    vectors = []

    # Iterate through the tokens and get their vectors
    for tok in tokens:
        if tok in ft_model.wv:
            vectors.append(ft_model.wv[tok])
        else:
            # If the token is not in the vocabulary, use the FastText model to get its vector
            vectors.append(ft_model.wv.get_vector(tok))

    if not vectors:
        return np.zeros(ft_model.vector_size)
    # If use_sum is True, sum the vectors of the tokens
    # Otherwise, return the mean vector
    vectors = np.array(vectors)
    return np.sum(vectors, axis=0) if use_sum else np.mean(vectors, axis=0)


An example of use with a toy code solution

In [ ]:
code_example = """
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
"""
# Using the mean of the vectors as solution vector
embedding = solution_to_vector(code_example, ft_model_code, use_sum=False)
print("Vector shape:", embedding.shape)
print("Vector:", embedding)


Now use the other data frame containing complexities, it's the same used in the previous section of analysis.

In [ ]:
import pandas as pd
df_complexity_labels_light_we = pd.DataFrame(df_complexity_labels_light)
df_complexity_labels_light_we.head()

Just to have an overview of the its structure

In [ ]:
df_complexity_labels_light_we.head(15)

,problem_id,problem_name,solution_id,time_complexity_inferred,space_complexity_inferred,time_curve_coefficient,space_curve_coefficient
0,0,339_C. Xenia and Weights,0_0,O(1),O(n**2),1.886800e-05,0.012433
1,0,339_C. Xenia and Weights,0_1,None,None,NaN,NaN
2,0,339_C. Xenia and Weights,0_2,O(1),O(1),4.261500e-06,584.000000
3,0,339_C. Xenia and Weights,0_3,None,None,NaN,NaN
4,0,339_C. Xenia and Weights,0_4,O(1),O(1),1.299450e-05,323.000000
5,0,339_C. Xenia and Weights,0_5,None,None,NaN,NaN
6,0,339_C. Xenia and Weights,0_6,O(1),O(1),6.575000e-06,283.000000
7,0,339_C. Xenia and Weights,0_7,None,None,NaN,NaN
8,0,339_C. Xenia and Weights,0_8,O(1),O(1),3.838500e-06,568.000000
9,0,339_C. Xenia and Weights,0_9,None,None,NaN,NaN


This block of code selects a balanced subset of problems based on their inferred time complexity labels (e.g., O(n), O(logn), etc.).
Steps Performed:
- Define a list of target complexity tags.
- For each tag:
    - Select problems labeled with that complexity.
    -  Ensure only one solution per problem is used (drop_duplicates on problem_id).
    - Randomly sample 20 problems if available; otherwise, use all the available ones.
    - Tag each sampled row with its corresponding label.
- Concatenate all the sampled subsets into a single DataFrame.

In [ ]:
import pandas as pd

# Selected tags for sampling
selected_tags = ["O(n)", "O(logn)", "O(n**2)", "O(1)"]
samples = []

# Iterate through the selected tags and sample solutions with those complexity labels
for tag in selected_tags:
    # Filter the rows with the specific tag
    df_tag = df_complexity_labels_light_we[
        df_complexity_labels_light_we["time_complexity_inferred"].apply(
            lambda t: t == tag if isinstance(t, str) else False
        )
    ]
    # Remove duplicate solutions for the same problem (e.g., 'problem_id')
    df_tag_unique = df_tag.drop_duplicates(subset="problem_id")

    # If there are at least 20 distinct problems with that tag, sample them randomly
    # Otherwise, take all the available ones
    if len(df_tag_unique) >= 20:
        df_sample = df_tag_unique.sample(n=20, random_state=42)
    else:
        df_sample = df_tag_unique.copy()

    # Add the selected tag to the samples
    df_sample["selected_tag"] = tag
    # Append all the samples in a unified list of dataframes
    samples.append(df_sample)
    print(f"Number of problems with tag '{tag}': {len(df_sample)}")

# Concatenate all samples into a single DataFrame
df_solution_selected = pd.concat(samples, ignore_index=True)


Number of problems with tag 'O(n)': 20
Number of problems with tag 'O(logn)': 20
Number of problems with tag 'O(n**2)': 20
Number of problems with tag 'O(1)': 20


Show the selected solution with the specified time complexities

In [110]:
df_solution_selected

,problem_id,problem_name,solution_id,time_complexity_inferred,space_complexity_inferred,time_curve_coefficient,space_curve_coefficient,selected_tag
0,1950,697_B. Barnicle,1950_2,O(n),O(n),1.296628e-06,2.992037,O(n)
1,1369,p02721 AtCoder Beginner Contest 161 - Yutori,1369_7,O(n),O(n+m),2.604247e-06,165.981206,O(n)
2,1052,583_A. Asphalting Roads,1052_4,O(n),O(n),7.718780e-07,195.720170,O(n)
3,528,1251_B. Binary Palindromes,528_0,O(n),O(n),4.052476e-05,267.012714,O(n)
4,1643,1301_B. Motarack's Birthday,1643_333,O(n),O(n**2),8.154746e-08,0.000190,O(n)
...,...,...,...,...,...,...,...,...
75,1941,1473_A. Replacing Elements,1941_82,O(1),O(1),1.405000e-07,59.000000,O(1)
76,3000,p02753 AtCoder Beginner Contest 158 - Station ...,3000_1,O(1),O(1),1.049500e-06,54.000000,O(1)
77,1451,p00444 Change,1451_0,O(1),O(1),3.316500e-06,106.000000,O(1)
78,1760,554_C. Kyoya and Colored Balls,1760_14,O(1),O(n),3.722313e-03,25.613269,O(1)


Now we need to retrieve the solution code of each of these selected rows...

In [ ]:
# Just to have a look to the original dataframe containing also the solutions
df_problem_and_human_solutions_list_we.head(10)

,problem_id,problem_name,description,correct_solution_list,data_source,source_specific_limits,codeforces_specific_metadata,tests,human_accuracy_rate,dataclass,complexity_framework,description_vector,cf_tags
0,0,339_C. Xenia and Weights,Xenia has a set of weights and pan scales. Eac...,"[{'solution_id': '0_0', 'solution_code': '__au...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 339, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '0000000101 3 ', '...",0.281633,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.40579710144927...,"[40.88922, 68.51662, 6.8400073, -3.7496269, 10...","[constructive algorithms, dfs and similar, dp,..."
1,1,1547_E. Air Conditioners,On a strip of land of length n there are k air...,"[{'solution_id': '1_0', 'solution_code': 'def ...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 1547, 'cf_index': 'E', 'cf_p...",{'public_tests': [{'input': '5 6 2 2 5 14 16 ...,0.620833,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.11409395973154...,"[-10.14601, 193.53925, 24.337927, 79.53583, 23...","[data structures, dp, implementation, shortest..."
2,2,268_C. Beautiful Sets of Points,Manao has invented a new mathematical term — a...,"[{'solution_id': '2_0', 'solution_code': 'if _...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 268, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '2 2 ', 'output': ...",0.525066,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.05527638190954...,"[-6.0463147, 75.508385, 18.788363, 13.632879, ...","[constructive algorithms, implementation]"
3,3,478_C. Table Decorations,"You have r red, g green and b blue balloons. T...","[{'solution_id': '3_0', 'solution_code': 'a = ...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 478, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '1 1 1 ', 'output'...",0.562264,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.41610738255033...,"[5.330355, 36.27579, 7.763661, 2.4407022, 53.7...",[greedy]
4,4,5_C. Longest Regular Bracket Sequence,This is yet another problem dealing with regul...,"[{'solution_id': '4_0', 'solution_code': 'stri...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 5, 'cf_index': 'C', 'cf_poin...","{'public_tests': [{'input': ')((())))(()()) ',...",0.395939,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.03846153846153...,"[-20.027811, 24.79554, 7.7614923, 13.442029, 3...","[constructive algorithms, data structures, dp,..."
5,5,622_A. Infinite Sequence,"Consider the infinite sequence of integers: 1,...","[{'solution_id': '5_0', 'solution_code': 'n = ...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 622, 'cf_index': 'A', 'cf_po...","{'public_tests': [{'input': '56 ', 'output': '...",0.615142,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.01538461538461...,"[-9.750153, 51.01206, 4.732046, 15.7149105, 52...","[implementation, math]"
6,6,997_B. Roman Digits,Let's introduce a number system which is based...,"[{'solution_id': '6_0', 'solution_code': 'def ...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 997, 'cf_index': 'B', 'cf_po...","{'public_tests': [{'input': '2 ', 'output': '1...",0.524590,{'dataclass_code': 'import sys import time imp...,"{'time_complexity_fail_rate': 0.09375, 'space_...","[-6.8545666, 65.14403, 16.192327, 18.872938, 6...","[brute force, combinatorics, dp, greedy, math]"
7,7,805_C. Find Amir,A few years ago Sajjad left his school and reg...,"[{'solution_id': '7_0', 'solution_code': 'prin...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 805, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '10 ', 'outp

This code retrieves the actual Python code of selected solutions based on their problem_id and solution_id.   
The goal is to:
- Match each entry in df_solution_selected with its corresponding full solution from the original dataset (df_problem_and_human_solutions_list_we).
- Extract the solution_code field from the correct solution.

In [ ]:
# List to store the extracted solution code strings
solutions_code = []
# Loop through each row of the selected solutions DataFrame
for index, row in df_solution_selected.iterrows():
    
    # Extract the problem and solution IDs
    problem_id = row['problem_id']
    solution_id = row['solution_id']

    # Filter the full dataset to find the row with the matching problem ID
    problem_row = df_problem_and_human_solutions_list_we[df_problem_and_human_solutions_list_we["problem_id"] == problem_id]

    if problem_row.empty:
        # Skip this iteration if the problem is not found
        print(f"Problem ID {problem_id} not found in the full dataset.")
        continue
    
    # Extract the list of correct solutions for this problem
    solution_list = problem_row["correct_solution_list"].values[0]
    
    # Find the specific solution matching the solution_id 
    selected_solution = None
    for s in solution_list:
        # If solution_list is a list of dicts or custom structure, filter accordingly
        if isinstance(s, dict) and s.get("solution_id") == solution_id:
            selected_solution = s
            break
    
    if selected_solution is None:
        print(f"Solution ID {solution_id} not found in problem ID {problem_id}.")
        continue

    # Extract the solution code and append it to the list
    solutions_code.append(selected_solution['solution_code'])


This lines binds the extracted raw Python code back into the DataFrame, containing the selected solutions with their time complexities and associating each solution with its actual implementation.

In [ ]:
# Add the list of extracted solution codes as a new column in the DataFrame
df_solution_selected["solution_code"] = solutions_code
# Just to see the new column introduced
df_solution_selected

,problem_id,problem_name,solution_id,time_complexity_inferred,space_complexity_inferred,time_curve_coefficient,space_curve_coefficient,selected_tag,solution_code
0,1950,697_B. Barnicle,1950_2,O(n),O(n),1.296628e-06,2.992037,O(n),"a,b=input().split('e')\nb=int(b)\n# print(a,b)..."
1,1369,p02721 AtCoder Beginner Contest 161 - Yutori,1369_7,O(n),O(n+m),2.604247e-06,165.981206,O(n),"n,k,c=map(int,input().split())\ns=list(input()..."
2,1052,583_A. Asphalting Roads,1052_4,O(n),O(n),7.718780e-07,195.720170,O(n),"n = int(input())\nA = [list(map(int, input().s..."
3,528,1251_B. Binary Palindromes,528_0,O(n),O(n),4.052476e-05,267.012714,O(n),Q = int(input())\nfor q in range(Q):\n n = ...
4,1643,1301_B. Motarack's Birthday,1643_333,O(n),O(n**2),8.154746e-08,0.000190,O(n),# by the authority of GOD author: manhar s...
...,...,...,...,...,...,...,...,...,...
75,1941,1473_A. Replacing Elements,1941_82,O(1),O(1),1.405000e-07,59.000000,O(1),def main():\n t = int(input())\n for x i...
76,3000,p02753 AtCoder Beginner Contest 158 - Station ...,3000_1,O(1),O(1),1.049500e-06,54.000000,O(1),"s=input()\nprint(""No"" if s==""AAA"" or s==""BBB"" ..."
77,1451,p00444 Change,1451_0,O(1),O(1),3.316500e-06,106.000000,O(1),while True:\n x = int(input())\n if x ==...
78,1760,554_C. Kyoya and Colored Balls,1760_14,O(1),O(n),3.722313e-03,25.613269,O(1),#!/usr/bin/python3\n\nimport sys\nfrom functoo...


It loops over each code solution and converts it into a vector using the solution_to_vector function (previously seen) and the pre-trained FastText model ft_model_code (it's the one trained with comments).
- solution_to_vector(...) tokenizes the code and retrieves embeddings for each token than aggregates them using the mean (use_sum=False) .


In [ ]:
vectors = []
for solution in df_solution_selected["solution_code"]:
    vectors.append(solution_to_vector(solution, ft_model_code, use_sum=False))

# Convert the list of vectors to a NumPy array
vectors = np.array(vectors)
# Check the shape of the array
print("Vectors Shape:",vectors.shape)  


Vectors Shape: (80, 100)


##### Visualization of Solution Embeddings with t-SNE

Plot the 3D projection of the solution vectors with different colors based on the time_complexity in the dataset, to see if there are some patterns based on it.

In [117]:
from sklearn.manifold import TSNE
import numpy as np


tsne = TSNE(n_components=3, random_state=42, perplexity=10)
vectors_3d = tsne.fit_transform(vectors)

df_solution_selected[["x", "y", "z"]] = vectors_3d

import plotly.express as px

fig = px.scatter_3d(
    df_solution_selected,
    x="x", y="y", z="z",
    color="selected_tag",
    #text="problem_name",
    hover_data=["problem_name", "selected_tag"],
    title="3D Projection of Solutions by Time Complexity Tag"
)

fig.update_traces(marker=dict(size=5, opacity=0.8))
fig.show()

The resulting 3D scatter plot does not exhibit a clear separation among the time complexity groups. This suggests that:
- The embedding model might not fully capture structural/time-complexity-related semantics.
- Time complexity may not be linearly separable in the vector space.